In [1]:
import sys
import os
import subprocess

app_base_dir = '/home/eusobg/EUSO-SPB/euso-spb-patt-reco-v1'
if app_base_dir not in sys.path:
    sys.path.append(app_base_dir)

import re
# import collections
import numpy as np
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as psql
import getpass
import matplotlib as mpl
import argparse
import glob
# from tqdm import tqdm
import traceback
import hashlib

from utility_funtions import str2bool_argparse


def __check_agg():
    show_plots = False
    if '--show-plots' in sys.argv[1:]:
        args_parser = argparse.ArgumentParser(description='')
        args_parser.add_argument('--show-plots',type=str2bool_argparse,default=False,help='If true, plots are only showed in windows')
        args , _ = args_parser.parse_known_args(sys.argv[1:])
        show_plots = args.show_plots
    if not show_plots:
        mpl.use('Agg')


__check_agg()


mpl.rcParams['figure.dpi'] = 80

import matplotlib.pyplot as plt

#import ROOT

# import tool.npy_frames_visualization as npy_vis
import tool.acqconv
import data_analysis_utils
import supervised_classification as supc

Welcome to JupyROOT 6.10/00


In [2]:
import event_recognition_efficiency

In [3]:
con = con = pg.connect(dbname='eusospb_data', user='eusospb', host='localhost', password='1e20eVjemeuso')
cur = con.cursor()

In [4]:
labeled_events_columns = ['event_id', 'source_file_acquisition_full', 'source_file_trigger_full', 'global_gtu', 'packet_id', 'gtu_in_packet', 'num_gtu', 'num_triggered_pixels',
        'gtu_y_hough__peak_thr2_avg_phi', 'gtu_x_hough__peak_thr2_avg_phi', 'gtu_y_hough__peak_thr3_avg_phi', 'gtu_x_hough__peak_thr3_avg_phi',
        'trigg_x_y_hough__peak_thr1__max_cluster_counts_sum_width', 'trigg_gtu_x_hough__peak_thr1__max_cluster_counts_sum_width', 'trigg_gtu_y_hough__peak_thr1__max_cluster_counts_sum_width',
        'gtu_y_hough__peak_thr1__max_cluster_counts_sum_width', 'gtu_x_hough__peak_thr1__max_cluster_counts_sum_width', 'trigg_x_y_hough__peak_thr1__max_cluster_counts_sum_width', 'x_y_hough__peak_thr1__max_cluster_counts_sum_width',
        'trigg_x_y_hough__dbscan_num_clusters_above_thr1', 'trigg_gtu_y_hough__dbscan_num_clusters_above_thr1', 'trigg_gtu_x_hough__dbscan_num_clusters_above_thr1',
        'gtu_y_hough__dbscan_num_clusters_above_thr1', 'gtu_x_hough__dbscan_num_clusters_above_thr1', 'x_y_active_pixels_num'
#         'etruth_trueenergy', 'egeometry_pos_z'
                         ]
low_energy_in_pmt_all_query = supc.get_query__select_training_data__low_energy_in_pmt(columns=labeled_events_columns, limit=10000, offset=0)
low_energy_in_pmt_all = psql.read_sql(low_energy_in_pmt_all_query, con) # maybe too many gtu ?
# led_all = psql.read_sql(supc.get_query__select_training_data__led(columns=labeled_events_columns, limit=10000, offset=0), con)

In [5]:
len(low_energy_in_pmt_all)

7067

In [6]:
outdir = '/home/eusobg/EUSO-SPB/low_energy_cr'
os.makedirs(outdir, exist_ok=True)

In [12]:
event_recognition_efficiency.vis_events_df(low_energy_in_pmt_all, outdir, 'low_energy_cr', events_per_figure=50, max_figures=10)

#0	(0)
	ID: 87144
	pck: 63
	gtu_in_packet: 39
	num_gtu: 11
	num_triggered_pixels: 483
#1	(3)
	ID: 247771
	pck: 171
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 482
#2	(6)
	ID: 28703
	pck: 32
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 472
#3	(9)
	ID: 117458
	pck: 31
	gtu_in_packet: 40
	num_gtu: 12
	num_triggered_pixels: 465
#4	(12)
	ID: 133168
	pck: 127
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 448
#5	(15)
	ID: 225442
	pck: 103
	gtu_in_packet: 40
	num_gtu: 12
	num_triggered_pixels: 444
#6	(18)
	ID: 225283
	pck: 74
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 440
#7	(21)
	ID: 209345
	pck: 64
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 438
#8	(24)
	ID: 230452
	pck: 34
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 438
#9	(27)
	ID: 58936
	pck: 43
	gtu_in_packet: 40
	num_gtu: 11
	num_triggered_pixels: 435
#10	(30)
	ID: 203248
	pck: 2
	gtu_in_packet: 39
	num_gtu: 13
	num_triggered_pixels: 433
#11	(33)
	ID: 1153
	pck: 4
	gtu_in_p

#91	(123)
	ID: 4239
	pck: 35
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
#92	(126)
	ID: 4534
	pck: 12
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
#93	(129)
	ID: 4691
	pck: 24
	gtu_in_packet: 39
	num_gtu: 13
	num_triggered_pixels: 288
#94	(132)
	ID: 4751
	pck: 29
	gtu_in_packet: 39
	num_gtu: 13
	num_triggered_pixels: 288
#95	(135)
	ID: 4785
	pck: 33
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
#96	(138)
	ID: 5005
	pck: 18
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
#97	(141)
	ID: 5106
	pck: 27
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
#98	(144)
	ID: 5161
	pck: 31
	gtu_in_packet: 39
	num_gtu: 13
	num_triggered_pixels: 288
#99	(147)
	ID: 5262
	pck: 43
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
#100	(150)
	ID: 5362
	pck: 51
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
SAVING FIGURE: /home/eusobg/EUSO-SPB/low_energy_cr/low_energy_cr_50_100.png
#100	(0)
	ID: 5362
	pck: 51
	gtu_in_packet:

#181	(93)
	ID: 10959
	pck: 10
	gtu_in_packet: 39
	num_gtu: 13
	num_triggered_pixels: 288
#182	(96)
	ID: 11055
	pck: 23
	gtu_in_packet: 39
	num_gtu: 13
	num_triggered_pixels: 288
#183	(99)
	ID: 11109
	pck: 32
	gtu_in_packet: 39
	num_gtu: 13
	num_triggered_pixels: 288
#184	(102)
	ID: 11164
	pck: 40
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
#185	(105)
	ID: 11216
	pck: 1
	gtu_in_packet: 39
	num_gtu: 13
	num_triggered_pixels: 288
#186	(108)
	ID: 11249
	pck: 5
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
#187	(111)
	ID: 11311
	pck: 12
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
#188	(114)
	ID: 11340
	pck: 18
	gtu_in_packet: 38
	num_gtu: 13
	num_triggered_pixels: 288
#189	(117)
	ID: 11400
	pck: 27
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
#190	(120)
	ID: 11423
	pck: 31
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
#191	(123)
	ID: 11538
	pck: 48
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 288
#192	(126)
	ID:

#269	(57)
	ID: 102874
	pck: 72
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 219
#270	(60)
	ID: 268904
	pck: 86
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 219
#271	(63)
	ID: 118894
	pck: 142
	gtu_in_packet: 39
	num_gtu: 11
	num_triggered_pixels: 218
#272	(66)
	ID: 189140
	pck: 54
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 218
#273	(69)
	ID: 276614
	pck: 24
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 217
#274	(72)
	ID: 219787
	pck: 62
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 216
#275	(75)
	ID: 41183
	pck: 97
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 214
#276	(78)
	ID: 246206
	pck: 144
	gtu_in_packet: 39
	num_gtu: 13
	num_triggered_pixels: 214
#277	(81)
	ID: 251807
	pck: 142
	gtu_in_packet: 40
	num_gtu: 12
	num_triggered_pixels: 214
#278	(84)
	ID: 279275
	pck: 129
	gtu_in_packet: 40
	num_gtu: 11
	num_triggered_pixels: 214
#279	(87)
	ID: 61850
	pck: 40
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 213
#280	(90

#357	(21)
	ID: 26460
	pck: 11
	gtu_in_packet: 39
	num_gtu: 11
	num_triggered_pixels: 173
#358	(24)
	ID: 133327
	pck: 74
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 173
#359	(27)
	ID: 146876
	pck: 331
	gtu_in_packet: 40
	num_gtu: 11
	num_triggered_pixels: 173
#360	(30)
	ID: 266252
	pck: 41
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 173
#361	(33)
	ID: 114546
	pck: 126
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 172
#362	(36)
	ID: 59068
	pck: 25
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 171
#363	(39)
	ID: 89786
	pck: 83
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 171
#364	(42)
	ID: 214012
	pck: 138
	gtu_in_packet: 38
	num_gtu: 11
	num_triggered_pixels: 171
#365	(45)
	ID: 233269
	pck: 2
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 171
#366	(48)
	ID: 239307
	pck: 145
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 171
#367	(51)
	ID: 240612
	pck: 138
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 171
#368	(54)

#446	(138)
	ID: 101136
	pck: 90
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 141
#447	(141)
	ID: 142167
	pck: 65
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 141
#448	(144)
	ID: 1509
	pck: 9
	gtu_in_packet: 38
	num_gtu: 12
	num_triggered_pixels: 139
#449	(147)
	ID: 31580
	pck: 148
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 139
#450	(150)
	ID: 89209
	pck: 31
	gtu_in_packet: 40
	num_gtu: 11
	num_triggered_pixels: 139
SAVING FIGURE: /home/eusobg/EUSO-SPB/low_energy_cr/low_energy_cr_400_450.png
#450	(0)
	ID: 89209
	pck: 31
	gtu_in_packet: 40
	num_gtu: 11
	num_triggered_pixels: 139
#451	(3)
	ID: 225829
	pck: 51
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 139
#452	(6)
	ID: 46292
	pck: 181
	gtu_in_packet: 40
	num_gtu: 13
	num_triggered_pixels: 138
#453	(9)
	ID: 72433
	pck: 51
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 138
#454	(12)
	ID: 172410
	pck: 89
	gtu_in_packet: 39
	num_gtu: 11
	num_triggered_pixels: 138
#455	(15)
	ID: 149686
	pck: 

In [7]:
low_energy_in_pmt_all_7ec = low_energy_in_pmt_all[ low_energy_in_pmt_all['x_y_active_pixels_num'] > 256*7 ]

In [8]:
len(low_energy_in_pmt_all_7ec)

5527

In [9]:
event_recognition_efficiency.vis_events_df(low_energy_in_pmt_all_7ec, outdir, 'low_energy_cr_7EC', events_per_figure=50, max_figures=3) # max_figures=200

#0	(0)
	ID: 87144
	pck: 63
	gtu_in_packet: 39
	num_gtu: 11
	num_triggered_pixels: 483
#1	(3)
	ID: 247771
	pck: 171
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 482
#2	(6)
	ID: 28703
	pck: 32
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 472
#3	(9)
	ID: 117458
	pck: 31
	gtu_in_packet: 40
	num_gtu: 12
	num_triggered_pixels: 465
#4	(12)
	ID: 133168
	pck: 127
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 448
#5	(15)
	ID: 225442
	pck: 103
	gtu_in_packet: 40
	num_gtu: 12
	num_triggered_pixels: 444
#6	(18)
	ID: 225283
	pck: 74
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 440
#7	(21)
	ID: 209345
	pck: 64
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 438
#8	(24)
	ID: 230452
	pck: 34
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 438
#9	(27)
	ID: 58936
	pck: 43
	gtu_in_packet: 40
	num_gtu: 11
	num_triggered_pixels: 435
#10	(30)
	ID: 227170
	pck: 92
	gtu_in_packet: 40
	num_gtu: 12
	num_triggered_pixels: 414
#11	(33)
	ID: 187024
	pck: 81
	gtu_

#91	(123)
	ID: 88232
	pck: 795
	gtu_in_packet: 38
	num_gtu: 11
	num_triggered_pixels: 246
#92	(126)
	ID: 166157
	pck: 99
	gtu_in_packet: 39
	num_gtu: 10
	num_triggered_pixels: 246
#93	(129)
	ID: 34095
	pck: 7
	gtu_in_packet: 39
	num_gtu: 11
	num_triggered_pixels: 245
#94	(132)
	ID: 232192
	pck: 32
	gtu_in_packet: 39
	num_gtu: 11
	num_triggered_pixels: 245
#95	(135)
	ID: 187680
	pck: 18
	gtu_in_packet: 40
	num_gtu: 11
	num_triggered_pixels: 243
#96	(138)
	ID: 18003
	pck: 50
	gtu_in_packet: 40
	num_gtu: 11
	num_triggered_pixels: 239
#97	(141)
	ID: 198638
	pck: 48
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 239
#98	(144)
	ID: 109762
	pck: 47
	gtu_in_packet: 39
	num_gtu: 11
	num_triggered_pixels: 238
#99	(147)
	ID: 274280
	pck: 16
	gtu_in_packet: 40
	num_gtu: 12
	num_triggered_pixels: 238
#100	(150)
	ID: 43777
	pck: 338
	gtu_in_packet: 40
	num_gtu: 10
	num_triggered_pixels: 235
SAVING FIGURE: /home/eusobg/EUSO-SPB/low_energy_cr/low_energy_cr_7EC_50_100.png
#100	(0)
	ID: 43777
	p

In [35]:
low_energy_in_pmt_all_7ec

,event_id,source_file_acquisition_full,source_file_trigger_full,global_gtu,packet_id,gtu_in_packet,num_gtu,num_triggered_pixels,gtu_y_hough__peak_thr2_avg_phi,gtu_x_hough__peak_thr2_avg_phi,...,gtu_y_hough__peak_thr1__max_cluster_counts_sum_width,gtu_x_hough__peak_thr1__max_cluster_counts_sum_width,trigg_x_y_hough__peak_thr1__max_cluster_counts_sum_width,x_y_hough__peak_thr1__max_cluster_counts_sum_width,trigg_x_y_hough__dbscan_num_clusters_above_thr1,trigg_gtu_y_hough__dbscan_num_clusters_above_thr1,trigg_gtu_x_hough__dbscan_num_clusters_above_thr1,gtu_y_hough__dbscan_num_clusters_above_thr1,gtu_x_hough__dbscan_num_clusters_above_thr1,x_y_active_pixels_num
0,87144,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,8103,63,39,11,483,-0.010349,0.010136,...,11.0,9.0,13.0,12.0,2,1,1,1.0,1.0,2290
1,247771,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,21928,171,40,10,482,-0.000293,0.010426,...,10.0,10.0,18.0,15.0,1,1,1,1.0,1.0,2297
2,28703,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,4136,32,40,10,472,-0.017651,-0.014080,...,11.0,9.0,32.0,15.0,2,1,1,1.0,1.0,2294
3,117458,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,4008,31,40,12,465,-0.000198,-0.012352,...,12.0,12.0,7.0,30.0,1,1,1,1.0,1.0,2292
4,133168,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,16295,127,39,10,448,-0.022676,-0.013946,...,10.0,9.0,13.0,9.0,1,1,1,1.0,1.0,2290
5,225442,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,13224,103,40,12,444,-0.011757,-0.002813,...,10.0,10.0,40.0,12.0,2,1,1,1.0,1.0,2290
6,225283,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,9512,74,40,10,440,-0.000452,-0.017419,...,11.0,9.0,22.0,14.0,1,1,1,1.0,1.0,2292
7,209345,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,8231,64,39,10,438,-0.000224,-0.007618,...,11.0,10.0,43.0,17.0,1,1,1,1.0,1.0,2291
8,230452,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,4391,34,39,10,438,-0.014281,-0.007690,...,9.0,10.0,19.0,12.0,2,1,1,1.0,1.0,2292
9,58936,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,5544,43,40,11,435,0.011060,-0.003433,...,8.0,10.0,26.0,21.0,1,2,1,2.0,1.0,2292


In [36]:
low_energy_in_pmt_all

,event_id,source_file_acquisition_full,source_file_trigger_full,global_gtu,packet_id,gtu_in_packet,num_gtu,num_triggered_pixels,gtu_y_hough__peak_thr2_avg_phi,gtu_x_hough__peak_thr2_avg_phi,...,gtu_y_hough__peak_thr1__max_cluster_counts_sum_width,gtu_x_hough__peak_thr1__max_cluster_counts_sum_width,trigg_x_y_hough__peak_thr1__max_cluster_counts_sum_width,x_y_hough__peak_thr1__max_cluster_counts_sum_width,trigg_x_y_hough__dbscan_num_clusters_above_thr1,trigg_gtu_y_hough__dbscan_num_clusters_above_thr1,trigg_gtu_x_hough__dbscan_num_clusters_above_thr1,gtu_y_hough__dbscan_num_clusters_above_thr1,gtu_x_hough__dbscan_num_clusters_above_thr1,x_y_active_pixels_num
0,87144,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,8103,63,39,11,483,-0.010349,1.013590e-02,...,11.0,9.0,13.0,12.0,2,1,1,1.0,1.0,2290
1,247771,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,21928,171,40,10,482,-0.000293,1.042566e-02,...,10.0,10.0,18.0,15.0,1,1,1,1.0,1.0,2297
2,28703,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,4136,32,40,10,472,-0.017651,-1.407998e-02,...,11.0,9.0,32.0,15.0,2,1,1,1.0,1.0,2294
3,117458,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,4008,31,40,12,465,-0.000198,-1.235157e-02,...,12.0,12.0,7.0,30.0,1,1,1,1.0,1.0,2292
4,133168,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,16295,127,39,10,448,-0.022676,-1.394575e-02,...,10.0,9.0,13.0,9.0,1,1,1,1.0,1.0,2290
5,225442,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,13224,103,40,12,444,-0.011757,-2.812838e-03,...,10.0,10.0,40.0,12.0,2,1,1,1.0,1.0,2290
6,225283,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,9512,74,40,10,440,-0.000452,-1.741921e-02,...,11.0,9.0,22.0,14.0,1,1,1,1.0,1.0,2292
7,209345,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,8231,64,39,10,438,-0.000224,-7.618000e-03,...,11.0,10.0,43.0,17.0,1,1,1,1.0,1.0,2291
8,230452,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,4391,34,39,10,438,-0.014281,-7.690025e-03,...,9.0,10.0,19.0,12.0,2,1,1,1.0,1.0,2292
9,58936,/media/node15_data2/SPBDATA/allpackets-SPBEUSO...,/home/eusobg/EUSO-SPB/SPBDATA_processed/allpac...,5544,43,40,11,435,0.011060,-3.432526e-03,...,8.0,10.0,26.0,21.0,1,2,1,2.0,1.0,2292


In [37]:
low_energy_in_pmt_all_7ec.to_csv('/home/eusobg/EUSO-SPB/euso-spb-patt-reco-v1/static/one_gtu_blips/data_7ec.csv', sep='\t')
low_energy_in_pmt_all.to_csv('/home/eusobg/EUSO-SPB/euso-spb-patt-reco-v1/static/one_gtu_blips/data_all.csv', sep='\t')

In [38]:
print( low_energy_in_pmt_all_query )


SELECT event_id, source_file_acquisition_full, source_file_trigger_full, global_gtu, packet_id, gtu_in_packet, num_gtu, num_triggered_pixels, gtu_y_hough__peak_thr2_avg_phi, gtu_x_hough__peak_thr2_avg_phi, gtu_y_hough__peak_thr3_avg_phi, gtu_x_hough__peak_thr3_avg_phi, trigg_x_y_hough__peak_thr1__max_cluster_counts_sum_width, trigg_gtu_x_hough__peak_thr1__max_cluster_counts_sum_width, trigg_gtu_y_hough__peak_thr1__max_cluster_counts_sum_width, gtu_y_hough__peak_thr1__max_cluster_counts_sum_width, gtu_x_hough__peak_thr1__max_cluster_counts_sum_width, trigg_x_y_hough__peak_thr1__max_cluster_counts_sum_width, x_y_hough__peak_thr1__max_cluster_counts_sum_width, trigg_x_y_hough__dbscan_num_clusters_above_thr1, trigg_gtu_y_hough__dbscan_num_clusters_above_thr1, trigg_gtu_x_hough__dbscan_num_clusters_above_thr1, gtu_y_hough__dbscan_num_clusters_above_thr1, gtu_x_hough__dbscan_num_clusters_above_thr1, x_y_active_pixels_num FROM spb_processing_event_ver2
    WHERE   
            source_data_typ